<a href="https://colab.research.google.com/github/karmanandan/Deep_learning_projects/blob/master/Spacy3_new_0822.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# https://towardsdatascience.com/improving-the-ner-model-with-patent-texts-spacy-prodigy-and-a-bit-of-magic-44c86282ea99
# https://spacy.io/usage/training#quickstart
# https://github.com/kinivi/patent_ner_linking/blob/main/project.ipynb

In [2]:
!pip install spacy-transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 53 kB 1.2 MB/s 
     |████████████████████████████████| 1.1 MB 10.4 MB/s 
     |████████████████████████████████| 4.7 MB 44.5 MB/s 
     |████████████████████████████████| 120 kB 59.6 MB/s 
     |████████████████████████████████| 6.6 MB 54.3 MB/s 


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import os
os.chdir('/content/drive/MyDrive/Spacy3')

In [5]:
%%writefile base_config.cfg
# This is an auto-generated partial config. To use it with 'spacy train'
# you can run spacy init fill-config to auto-fill all default settings:
# python -m spacy init fill-config ./base_config.cfg ./config.cfg
[paths]
train = null
dev = null
vectors = null
[system]
gpu_allocator = "pytorch"

[nlp]
lang = "en"
pipeline = ["transformer","ner"]
batch_size = 1000
max_length = 20000

[components]

[components.transformer]
factory = "transformer"

[components.transformer.model]
@architectures = "spacy-transformers.TransformerModel.v3"
name = "roberta-base"
tokenizer_config = {"use_fast": true}

[components.transformer.model.get_spans]
@span_getters = "spacy-transformers.strided_spans.v1"
window = 128
stride = 96

[components.ner]
factory = "ner"

[components.ner.model]
@architectures = "spacy.TransitionBasedParser.v2"
state_type = "ner"
extra_state_tokens = false
hidden_width = 64
maxout_pieces = 2
use_upper = false
nO = null

[components.ner.model.tok2vec]
@architectures = "spacy-transformers.TransformerListener.v1"
grad_factor = 1.0

[components.ner.model.tok2vec.pooling]
@layers = "reduce_mean.v1"

[corpora]

[corpora.train]
@readers = "spacy.Corpus.v1"
path = ${paths.train}
max_length = 0

[corpora.dev]
@readers = "spacy.Corpus.v1"
path = ${paths.dev}
max_length = 0

[training]
accumulate_gradient = 3
dev_corpus = "corpora.dev"
train_corpus = "corpora.train"

[training.optimizer]
@optimizers = "Adam.v1"

[training.optimizer.learn_rate]
@schedules = "warmup_linear.v1"
warmup_steps = 250
total_steps = 20000
initial_rate = 5e-5

[training.batcher]
@batchers = "spacy.batch_by_padded.v1"
discard_oversize = true
size = 2000
buffer = 256

[initialize]
vectors = ${paths.vectors}

Overwriting base_config.cfg


In [6]:
# !7z x 'G06K.txt.gz'

In [7]:
!ls

base_config.cfg  G06K.txt.gz	      spacy_output
config.cfg	 manyterms.lower.txt  training_data.spacy
G06K.txt	 ner_patent_g06k      valid_data.spacy


In [8]:
!python -m spacy download en_core_web_lg

2022-08-25 13:14:18.646870: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 587.7 MB 8.8 kB/s 
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')


In [9]:
import sys
import time
import json
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
import spacy
from spacy.matcher import PhraseMatcher, Matcher
from spacy.util import filter_spans
from spacy import displacy
from spacy.tokens import DocBin
from spacy.tokens import Span
from collections import Counter
from nltk.tokenize import MWETokenizer
from nltk.util import Trie
import nltk
from nltk.corpus import wordnet as wn
nltk.download('wordnet')
nltk.download('omw-1.4')
tqdm.pandas()
spacy.__version__

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


'3.4.1'

In [10]:
# if you've already unzipped the file
patent_data = open('G06K.txt').read().strip()

# split into patents texts | 1 entry = 1 patent
patent_texts = patent_data.split('\n\n')

# split each patent into lines
patent_lines = patent_data.split('\n')

In [11]:
patent_lines[0:10]

['COMMUNICATION DEVICE, COMMUNICATION METHOD AND PROGRAM',
 '_____2019_____3500050_____490084061_____EP3500000.txt_____G06K_____G06K7/10722:G06K7/1417:H04L67/104:H04M1/00:H04M11/00:H04W12/001:H04W12/04:H04W12/04033:H04W12/04071:H04W12/06:H04W76/14:H04W84/12:H04W84/20',
 'A communication device obtains identification information and a public key of a first other communication device by a particular obtaining method that does not use a wireless LAN and notifies the first other communication device of a role of the first other communication device in a communication based on Wi-Fi Direct. In addition, the communication device obtains identification information and a public key of a second other communication device by the particular obtaining method and notifies the second other communication device of a role of the second other communication device in the communication based on Wi-Fi Direct. One of the notified roles is a P2P Group Owner and the other one is a P2P Client, and the communi

In [12]:
# here are the potential terms
mwes = open('manyterms.lower.txt').read().lower().strip().split('\n')
print(mwes[44444:44456])
print(len(mwes), 'mwes')

['antonio superchi', 'antonio tarver', 'antonio torres jurado', 'antonio valdes', 'antonio valdes y fernandez bazan', 'antonio valdez', 'antonio valdés y bazán', 'antonio valdés y fernández bazán', 'antonio valente', 'antonio vitali', 'antonio vivaldi', 'antonio xavier machado e cerveira']
743274 mwes


In [13]:
# Here lowercase=False option is used to keep the original case of the terms, since we possibly could have term abbreviations. Like API, CAT, etc.
cvectorizer = CountVectorizer(ngram_range=(
    1, 4), stop_words="english", vocabulary=mwes, lowercase=True)
X = cvectorizer.fit_transform(patent_texts)

# Show top-25 most frequent terms
termdf_cv = pd.DataFrame(np.sum(X, axis=0), columns=cvectorizer.get_feature_names(
)).T.sort_values(by=0, ascending=False)
termdf_cv.head(25)

/usr/local/lib/python3.7/dist-packages/sklearn/feature_extraction/text.py:1323: UserWarning: Upper case characters found in vocabulary while 'lowercase' is True. These entries will not be matched with any documents
  "Upper case characters found in"
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,0
electronic device,16280
image processing,12224
control unit,9263
mobile terminal,9165
information processing,7732
neural network,6734
user interface,6177
computer readable,6103
fingerprint sensor,5980
display device,5666


In [14]:
nlp = spacy.load("en_core_web_lg")
doc = nlp(patent_texts[0][18000:20000])
displacy.render(doc, style="ent", jupyter=True)

In [15]:
matcher = PhraseMatcher(nlp.vocab, attr="LOWER")
patterns = [nlp.make_doc(text) for text in termdf_cv.index]

In [16]:
matcher

In [17]:
matcher.add("Tech", patterns)

In [18]:
train_lines, test_lines = train_test_split(
    patent_lines, test_size=0.3, random_state=42)

In [19]:
def create_dataset(text, n_lines, filename, offset=0):
    LABEL = "TECH"
    doc_bin = DocBin()  # create a DocBin object

    for training_example in tqdm(text[offset:offset+n_lines]):
        doc = nlp.make_doc(training_example)
        ents = []

        for match_id, start, end in matcher(doc):
            span = Span(doc, start, end, label=LABEL)
            if span is None:
                print("Skipping entity")
            else:
                ents.append(span)

        filtered_ents = filter_spans(ents)
        doc.ents = filtered_ents
        doc_bin.add(doc)
    doc_bin.to_disk(filename)

In [20]:
create_dataset(train_lines, 40_000, "training_data.spacy")
create_dataset(test_lines, 12_000, "valid_data.spacy")

  0%|          | 0/40000 [00:00<?, ?it/s]

  0%|          | 0/12000 [00:00<?, ?it/s]

In [21]:
!python -m spacy init fill-config base_config.cfg config.cfg

2022-08-25 13:18:09.676859: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
✔ Auto-filled config with all values
✔ Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [22]:
# !python -m spacy train config.cfg --output ./spacy_output --paths.train ./training_data.spacy --paths.dev ./valid_data.spacy --gpu-id 0

In [23]:
!git lfs install
!git clone https://huggingface.co/kinivi/ner_patent_g06k


Error: Failed to call git rev-parse --git-dir --show-toplevel: "fatal: not a git repository (or any parent up to mount point /content)\nStopping at filesystem boundary (GIT_DISCOVERY_ACROSS_FILESYSTEM not set).\n"
Git LFS initialized.
fatal: destination path 'ner_patent_g06k' already exists and is not an empty directory.


In [24]:
nlp = spacy.load("./ner_patent_g06k/spacy_output/model-best")

doc = nlp("Wi-Fi Direct (registered trademark, which will be hereinafter referred to as WFD) \
           corresponding to a technology for directly performing a communication based on a \
           wireless LAN between communication devices without intermediation of an access \
           point (hereinafter referred to as AP) is standardized in Wi-Fi Alliance serving \
           as a wireless LAN industry group.")

colors = {"TECH": "#F67DE3"}
options = {"colors": colors}

spacy.displacy.render(doc, style="ent", options=options, jupyter=True)

/usr/local/lib/python3.7/dist-packages/spacy/util.py:865: UserWarning: [W095] Model 'en_pipeline' (0.0.0) was trained with spaCy v3.2 and may not be 100% compatible with the current version (3.4.1). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)
/usr/local/lib/python3.7/dist-packages/torch/amp/autocast_mode.py:198: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')


In [25]:
nlp = spacy.load("./ner_patent_g06k/prodigy_output/model-best")

doc = nlp("Wi-Fi Direct (registered trademark, which will be hereinafter referred to as WFD) \
           corresponding to a technology for directly performing a communication based on a \
           wireless LAN between communication devices without intermediation of an access \
           point (hereinafter referred to as AP) is standardized in Wi-Fi Alliance serving \
           as a wireless LAN industry group.")

colors = {"TECH": "#F67DE3"}
options = {"colors": colors}

spacy.displacy.render(doc, style="ent", options=options, jupyter=True)

In [26]:
# !python -m prodigy

In [27]:
# !prodigy ner.correct fine_tune_g06k2 spacy_output/model-best G06K.txt --loader txt --label TECH